In [1]:
!pip3 install --upgrade --user google-generativeai

In [2]:
!pip install python-dotenv

In [3]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
import google.generativeai as genai
genai.configure(api_key=os.getenv('GCP_API_KEY'))

In [2]:
model = genai.GenerativeModel("gemini-pro")

In [4]:
responses = model.generate_content("Why should I move to the cloud from on-premises deployments?", stream=False)

for response in responses:
    print(response.text, end="")

1. **Cost Savings**:
   - Reduced hardware and infrastructure costs: No need to purchase, maintain, and upgrade physical servers.
   - Reduced IT staff costs: Cloud providers handle infrastructure management, freeing up IT staff for other tasks.
   - Utility-based pricing: Pay only for the resources you use, eliminating upfront capital investments.
   - Reduced power and cooling costs: Data centers are designed for energy efficiency.


2. **Scalability**:
   - Easily scale up or down based on demand: Add or remove resources quickly to meet changing needs.
   - Handle sudden traffic spikes without performance degradation: Cloud infrastructure is designed to handle large volumes.


3. **Reliability**:
   - High availability and fault tolerance: Cloud providers have redundant systems and backup strategies to ensure uptime.
   - Disaster recovery: Data is replicated across multiple data centers for protection against disasters.


4. **Security**:
   - Cloud providers have dedicated securit

### Working with Model Parameters

In [12]:
generation_config=genai.types.GenerationConfig(
    temperature=0.5,
    top_p=0.5,
    top_k=32,
    candidate_count=1,
    max_output_tokens=500,
)

In [ ]:
responses = model.generate_content(
    "Why should I move to the cloud from on-premises deployments?",
    generation_config=generation_config,
    stream=True,
)

for response in responses:
    print(response.text, end="")

### Chat Prompts with Gemini

In [15]:
chat = model.start_chat()

prompt = """You are a book expert. I love reading books on leadership. Recommend me some based on that.
"""

responses = chat.send_message(prompt, stream=True)

for response in responses:
    print(response.text, end="")

1. **"Leadership in War: Essential Principles from History's Greatest Strategists" by Andrew Roberts**: An exploration of leadership through the lives of great military leaders, including Alexander the Great, Julius Caesar, and Napoleon Bonaparte.

2. **"The 7 Habits of Highly Effective People" by Stephen Covey**: A classic guide to self-improvement and leadership, emphasizing the importance of integrity, responsibility, and personal growth.

3. **"Good to Great: Why Some Companies Make the Leap...and Others Don't" by Jim Collins**: A study of companies that made the transition from good to great, identifying the key factors that contributed to their success.

4. **"Emotional Intelligence 2.0" by Travis Bradberry and Jean Greaves**: A practical guide to understanding and managing your emotions, with a focus on developing emotional intelligence for effective leadership.

5. **"The Five Dysfunctions of a Team: A Leadership Fable" by Patrick Lencioni**: A fictional story that illustrates 

In [16]:
prompt = "Are these books all from the same publisher?"

responses = chat.send_message(prompt, stream=True)

for response in responses:
    print(response.text, end="")

No, the books I recommended are not all from the same publisher. They are published by a variety of different publishers, including:

* HarperCollins Leadership
* Free Press
* Portfolio
* TalentSmart
* Jossey-Bass
* Penguin Books
* Greenleaf Center for Servant-Leadership
* Harvard Business Review Press
* McGraw-Hill Education
* John Wiley & Sons

This reflects the diversity of perspectives and approaches to leadership that exist in the field. Each publisher has its own unique focus and style, and the books I selected represent a range of these perspectives.

I hope this helps!

In [17]:
print(chat.history)

[parts {
  text: "You are a book expert. I love reading books on leadership. Recommend me some based on that.\n"
}
role: "user"
, parts {
  text: "1. **\"Leadership in War: Essential Principles from History\'s Greatest Strategists\" by Andrew Roberts**: An exploration of leadership through the lives of great military leaders, including Alexander the Great, Julius Caesar, and Napoleon Bonaparte.\n\n2. **\"The 7 Habits of Highly Effective People\" by Stephen Covey**: A classic guide to self-improvement and leadership, emphasizing the importance of integrity, responsibility, and personal growth.\n\n3. **\"Good to Great: Why Some Companies Make the Leap...and Others Don\'t\" by Jim Collins**: A study of companies that made the transition from good to great, identifying the key factors that contributed to their success.\n\n4. **\"Emotional Intelligence 2.0\" by Travis Bradberry and Jean Greaves**: A practical guide to understanding and managing your emotions, with a focus on developing emot

### Generating Code with Gemini

In [23]:
generation_config=genai.types.GenerationConfig(
    temperature=0.5,
    top_p=0.5,
    top_k=32,
    candidate_count=1,
    max_output_tokens=1000,
)

In [24]:
prompt = """Generate me terraform code to deploy based on the following specifications.
1. Cloud Provider: Google Cloud Platform (GCP)
2. Resources:
- Compute Engine instances: 3 n1-standard-1 instances
- Cloud Storage: 1 bucket for backups
- VPC: Custom network with 2 subnets in different regions
3. Networking:
- Cloud NAT for private subnet
- Firewall rules allowing HTTP and internal traffic
4. Security: IAM roles for Compute Engine access
5. Database: Cloud SQL instance - PostgreSQL, db-custom-2-7680
6. Regions: europe-west1 and us-central1 for the two subnets
7. Auto-scaling: Managed instance group for Compute Engine with auto-scaling enabled
8. Terraform State: Remote backend using GCP Cloud Storage
9. Terraform Version: v0.15.1
"""

In [25]:
responses = model.generate_content(
    prompt,
    generation_config=generation_config,
    stream=True,
)

In [26]:
for response in responses:
    print(response.text, end="")

```
# Define the Google Cloud Platform provider
provider "google" {
  project = "your-project-id"
  region  = "europe-west1"
}

# Create a custom VPC network
resource "google_compute_network" "vpc" {
  name = "custom-vpc"
}

# Create two subnets in different regions
resource "google_compute_subnetwork" "subnet-europe" {
  name          = "europe-subnet"
  region        = "europe-west1"
  network       = google_compute_network.vpc.id
  ip_cidr_range = "10.0.0.0/24"
}

resource "google_compute_subnetwork" "subnet-us" {
  name          = "us-subnet"
  region        = "us-central1"
  network       = google_compute_network.vpc.id
  ip_cidr_range = "10.1.0.0/24"
}

# Create a Cloud NAT gateway for the private subnet
resource "google_compute_router" "nat-gateway" {
  name            = "nat-gateway"
  network         = google_compute_network.vpc.id
  region          = "us-central1"
  nats            = ["nat-config"]
}

resource "google_compute_router_nat" "nat-config" {
  name           = "nat